In [54]:
# Import dependencies and connect to the XRPL
import xrpl
import nest_asyncio
nest_asyncio.apply()
test_url = 'https://s.altnet.rippletest.net:51234'
client = xrpl.clients.JsonRpcClient(test_url)

In [55]:
# Stand-alone code sample for the "issue a token" tutorial:
# https://xrpl.org/issue-a-fungible-token.html
# License: https://github.com/XRPLF/xrpl-dev-portal/blob/master/LICENSE

# Connect ----------------------------------------------------------------------
import xrpl
testnet_url = "https://s.altnet.rippletest.net:51234"
client = xrpl.clients.JsonRpcClient(testnet_url)


# Get credentials from the Testnet Faucet --------------------------------------
# For production, instead create a Wallet instance
faucet_url = "https://faucet.altnet.rippletest.net/accounts"
print("Getting 2 new accounts from the Testnet faucet...")
from xrpl.wallet import generate_faucet_wallet
cold_wallet = generate_faucet_wallet(client, debug=True)
hot_wallet = generate_faucet_wallet(client, debug=True)


# Configure issuer (cold address) settings -------------------------------------
cold_settings_tx = xrpl.models.transactions.AccountSet(
    account=cold_wallet.classic_address,
    transfer_rate=0,
    tick_size=5,
    domain=bytes.hex("example.com".encode("ASCII")),
    set_flag=xrpl.models.transactions.AccountSetFlag.ASF_DEFAULT_RIPPLE,
)
cst_prepared = xrpl.transaction.safe_sign_and_autofill_transaction(
    transaction=cold_settings_tx,
    wallet=cold_wallet,
    client=client,
)
print("Sending cold address AccountSet transaction...")
response = xrpl.transaction.send_reliable_submission(cst_prepared, client)
print(response)


# Configure hot address settings -----------------------------------------------
hot_settings_tx = xrpl.models.transactions.AccountSet(
    account=hot_wallet.classic_address,
    set_flag=xrpl.models.transactions.AccountSetFlag.ASF_REQUIRE_AUTH,
)
hst_prepared = xrpl.transaction.safe_sign_and_autofill_transaction(
    transaction=hot_settings_tx,
    wallet=hot_wallet,
    client=client,
)
print("Sending hot address AccountSet transaction...")
response = xrpl.transaction.send_reliable_submission(hst_prepared, client)
print(response)


# Create trust line from hot to cold address -----------------------------------
currency_code = "XDA"
trust_set_tx = xrpl.models.transactions.TrustSet(
    account=hot_wallet.classic_address,
    limit_amount=xrpl.models.amounts.issued_currency_amount.IssuedCurrencyAmount(
        currency=currency_code,
        issuer=cold_wallet.classic_address,
        value="9000000000", # Large limit, arbitrarily chosen
    )
)
ts_prepared = xrpl.transaction.safe_sign_and_autofill_transaction(
    transaction=trust_set_tx,
    wallet=hot_wallet,
    client=client,
)
print("Creating trust line from hot address to issuer...")
response = xrpl.transaction.send_reliable_submission(ts_prepared, client)
print(response)


# Send token -------------------------------------------------------------------
issue_quantity = "500000"
send_token_tx = xrpl.models.transactions.Payment(
    account=cold_wallet.classic_address,
    destination=hot_wallet.classic_address,
    amount=xrpl.models.amounts.issued_currency_amount.IssuedCurrencyAmount(
        currency=currency_code,
        issuer=cold_wallet.classic_address,
        value=issue_quantity
    )
)
pay_prepared = xrpl.transaction.safe_sign_and_autofill_transaction(
    transaction=send_token_tx,
    wallet=cold_wallet,
    client=client,
)
print(f"Sending {issue_quantity} {currency_code} to {hot_wallet.classic_address}...")
response = xrpl.transaction.send_reliable_submission(pay_prepared, client)
print(response)


# Check balances ---------------------------------------------------------------
print("Getting hot address balances...")
response = client.request(xrpl.models.requests.AccountLines(
    account=hot_wallet.classic_address,
    ledger_index="validated",
))
print(response)

print("Getting cold address balances...")
response = client.request(xrpl.models.requests.GatewayBalances(
    account=cold_wallet.classic_address,
    ledger_index="validated",
    hotwallet=[hot_wallet.classic_address]
))
print(response)

Getting 2 new accounts from the Testnet faucet...
Attempting to fund address r3SsGiAgxMRX7UKUy1UnXsrS7JmJyv1v1P
Faucet fund successful.
Attempting to fund address rpXcMdWEkwNdDDjdShKtVkWxLh8NSPwdXQ
Faucet fund successful.
Sending cold address AccountSet transaction...
Response(status=<ResponseStatus.SUCCESS: 'success'>, result={'Account': 'r3SsGiAgxMRX7UKUy1UnXsrS7JmJyv1v1P', 'Domain': '6578616D706C652E636F6D', 'Fee': '10', 'Flags': 0, 'LastLedgerSequence': 29787763, 'Sequence': 29787738, 'SetFlag': 8, 'SigningPubKey': 'EDB23F4D22437B15BB4A06B68BE45B7889845045FE4A01E9B8C6E9288DD1EF932D', 'TickSize': 5, 'TransactionType': 'AccountSet', 'TransferRate': 0, 'TxnSignature': '896EFE2A8941B3FC812E6576413C87418306830B8C3111BE74E102449E2C7240E23EC5026D54CE74AAA7BEAB6FBEBC3320CF64A37142CE8532FBB26E2FC3E40C', 'date': 712119182, 'hash': '611A0D3EF0F3A4F5608B6B59B527F16CC8E3978508CE5A1D5C7BE58BB1E45A5E', 'inLedger': 29787745, 'ledger_index': 29787745, 'meta': {'AffectedNodes': [{'ModifiedNode': {'F

In [56]:
print(response)

Response(status=<ResponseStatus.SUCCESS: 'success'>, result={'account': 'r3SsGiAgxMRX7UKUy1UnXsrS7JmJyv1v1P', 'balances': {'rpXcMdWEkwNdDDjdShKtVkWxLh8NSPwdXQ': [{'currency': 'XDA', 'value': '500000'}]}, 'ledger_hash': 'A240BF8615817A23D5B88B38B1DDDD561D307EA94A09D788C0169DBD3B331AFD', 'ledger_index': 29787757, 'validated': True}, id=None, type=<ResponseType.RESPONSE: 'response'>)
